# Road Following - Data Collection (Mouse)

If you've run through the collision avoidance sample in Day 2, you should now be familiar with these four steps:

1. Data collection
2. Training
3. Optimizing
3. Deployment

In this notebook, we'll do the exact same thing! Except, instead of classification, you'll learn a different fundamental technique known as **regression**, which we'll use to enable our Jetbot to follow a path on the road.

This notebook is for **Data Collection** only and we will be performing the following tasks:

1. Display the live camera feed using a special ipywidget called jupyter clickable image widget
2. Then collect data by performing the following steps:
- Place the JetBot at different positions on a path (offset from center, different angles, etc)
- Using your mouse, place a 'green dot', which corresponds to the target direction we want the robot to travel on.
- Store the X, Y values of this green dot along with the image from the robot's camera

What we get, is a 'carrot on a stick' that moves along our desired trajectory.  Deep learning decides where to place the carrot, and JetBot just follows it :)

## Install Dependencies!
By default `jupyter_clickable_image_widget` is not installed on the Jetbot therefore we would need to first install it before continuing on with this notebook. To do that, go to your terminal and paste the following commands line by line:   
1) cd   
2) sudo apt-get install nodejs-dev node-gyp libssl1.0-dev   
3) sudo apt-get install npm    
4) git clone https://github.com/jaybdub/jupyter_clickable_image_widget   
5) cd jupyter_clickable_image_widget   
6) git checkout no_typescript   
7) sudo pip3 install -e .   
8) sudo jupyter labextension install js   
9) sudo jupyter lab build   

### Import Libraries

So lets get started by importing all the required libraries for "data collection" purpose. We will mainly use OpenCV to visualize and save image with labels. Libraries such as uuid, datetime are used for image naming. 

In [ ]:
# IPython Libraries for display and widgets
import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Basic python packages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

### Collect data

Firstly, let's initialize and display our camera like we did in the previous notebooks, however, this time with using a special ipywidget called `jupyter_clickable_image_widget` that lets you click on the image and take the coordinates for data annotation. This eliminates the needs of using the gamepad controller for data annotation.

Our neural network takes a 224x224 pixel image as input. We'll set our camera to that size to minimize the filesize of our dataset. In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.

The following block of code below will display the live image feed for you to click on for annotation on the left, as well as the snapshot of last annotated image (with a green circle showing where you clicked) on the right.     

When you click on the left live image, it stores a file in the ``dataset_xy`` folder with files named

``xy_<x value>_<y value>_<uuid>.jpg``

When we train, we load the images and parse the x, y values from the filename.
Here `<x value>` and `<y value>` are the coordinates **in pixels** (counted from the top left corner).

In [ ]:
from jupyter_clickable_image_widget import ClickableImageWidget

DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created because they already exist')

camera = Camera()

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
count_widget = ipywidgets.IntText(description='count')
# manually update counts at initialization
count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # save to disk
        uuid = 'xy_%03d_%03d_%s' % (x, y, uuid1())
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(camera_widget.value)
        
        # display saved snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
        
camera_widget.on_msg(save_snapshot)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    count_widget
])

display(data_collection_widget)

Now, let's close the camera connection properly so that we can use the camera in other notebooks.

In [ ]:
camera.stop()

## Save dataset in a zip (optional)

Once you've collected enough data, we can choose to copy that data to a GPU desktop or cloud machine for training. To do that, we can call the following *terminal* command to compress
our dataset folder into a single *zip* file.

> The ! prefix indicates that we want to run the cell as a *shell* (or *terminal*) command.

> The -r flag in the zip command below indicates *recursive* so that we include all nested files, the -q flag indicates *quiet* so that the zip command doesn't print any output

In [ ]:
!zip -r -q dataset_mouse.zip dataset_xy

You should see a file named ``dataset_mouse.zip`` appear in the Jupyter Lab file browser.

Next, Let's move on to the "train_model.ipynb" notebook so that we can start training our model on the collected dataset!